In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import path
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint, LineString, LinearRing
from shapely.ops import cascaded_union, unary_union, transform
import datetime
import math
from scipy.ndimage.interpolation import shift
import scipy.interpolate as si
import shapely.wkt
from shapely.validation import explain_validity,make_valid
import xarray as xr
import seaborn as sns
from my_functions import sat_vap_press, vap_press, hot_dry_windy, haines
from joblib import Parallel, delayed
import multiprocessing
from os.path import exists
import rasterio
from rasterio.windows import get_data_window,Window, from_bounds
from rasterio.plot import show
from itertools import product

from timezonefinder import TimezoneFinder
import pytz

In [19]:
#process features using my polygons
sit209=pd.read_csv('../Query2.txt')
sit209.columns = ['INC209R_IDENTIFIER','INCIDENT_NAME','REPORT_FROM_DATE','REPORT_TO_DATE',
              'RESTYP_IDENTIFIER', 'RESOURCE_QUANTITY','RESOURCE_PERSONNEL','CODE_NAME',
              'INC_IDENTIFIER','PCT_CONTAINED_COMPLETED'] 


#2020 fires
#fire_incidents = ['LAKE','DOLAN','BOBCAT','CREEK','AUGUST COMPLEX']
fire_incidents=['AUGUST COMPLEX']

path_poly = '/data2/lthapa/ML_daily/fire_polygons/'
suffix_poly = 'Z_day_start.geojson'
start_time=12
for jj in range(len(fire_incidents)):
    fire_name = fire_incidents[jj].lower().replace(' ','_')
    print(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    fire_daily = gpd.read_file(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    print(fire_daily.crs)
    fire_daily=fire_daily.drop(columns=['Current Overpass'])
    fire_daily = fire_daily.drop(np.where(fire_daily['geometry']==None)[0])
    fire_daily['fire area (ha)'] = fire_daily['geometry'].area/10000 #hectares. from m2
    fire_daily.set_geometry(col='geometry', inplace=True) #designate the geometry column
    fire_daily = fire_daily.rename(columns={'Current Day':'UTC Day', 'Local Day': str(start_time)+ 'Z Start Day'})
    
    fire_daily = fire_daily.iloc[np.array(fire_daily['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]
    
    #calculate fuel-weighted FWI
    fuel_loading_fwi = fuel_loading_fwi_timeseries(fire_daily.iloc[0:4],12)
    print(fuel_loading_fwi)
    fuel_loading_fwi.to_csv('./fire_features/'+fire_name+'_Daily_FUEL_FWI_'+str(start_time)+'Z_day_start.csv') #daily averages


/data2/lthapa/ML_daily/fire_polygons/august_complex_VIIRS_daily_12Z_day_start.geojson
epsg:3347


KeyboardInterrupt: 

In [13]:
def fuel_loading_fwi_timeseries(df,day_start_hour):
    loadings = pd.read_csv('FWI_Category_Fuel_Loadings.csv').set_index('VALUE') #we will index by the fuel type!
    """for ii in range(1):#(len(df))):
        print(ii)
        today_poly = df.iloc[ii:ii+1] #get the first one, to test it
        fire_fuel_intersection = calculate_loading_grid_intersection(today_poly,50, loadings)
        print(fire_fuel_intersection)"""
        
        
    tic = datetime.datetime.now()    
    fuel_loadings = Parallel(n_jobs=2)(delayed(calculate_loading_grid_intersection)
                                 (df.iloc[ii:ii+1],50, loadings) 
                                 for ii in range(len(df)))
    #df_loadings=pd.DataFrame(fuel_loadings)
    
    toc = datetime.datetime.now()
    print(toc-tic)
    df_loadings = pd.concat(fuel_loadings)
    return df_loadings
    

In [15]:
#poly is the polygon for one timestep (in lcc)
#grid is an xarray of the subset of the grid
#grid_names is a string array [0:'lat_center_name',1:'lon_center_name',2:'lat_corner_name',3:'lon_corner_name']

def calculate_loading_grid_intersection(poly,bf, loadings):
    #get the bounds of the buffered polygons
    poly_latlon =poly.to_crs(epsg=5070)
    bounds = poly_latlon.buffer(bf).bounds
    #print(bounds)
    
    left = bounds['minx'].values
    right = bounds['maxx'].values
    bottom=bounds['miny'].values
    top=bounds['maxy'].values
    
    print(left, bottom, top,right)
    
    #load in the subset of the geotiff
    with rasterio.open('../FCCS_Fuel_Fire_Danger_Metric.tif') as src:
        win = from_bounds(left,bottom,right,top, src.transform)
        #print(win)
        w_from_latlon = src.read(1, window=win)
        w_flat = w_from_latlon.flatten()
        
    #build the lat/lon vectors associated with the data we pulled    
    rows = np.arange(win.row_off,win.row_off+win.height-1)
    cols = np.arange(win.col_off,win.col_off+win.width-1)
    COLS,ROWS=np.meshgrid(cols,rows)
    xs, ys = rasterio.transform.xy(src.transform, ROWS, COLS)
    xs = np.array(xs)
    ys = np.array(ys)
    
    xs_corner, ys_corner = calculate_grid_cell_corners(xs,xs)
    
    grid = build_grid_xarray(ys_corner, xs_corner, ys, xs)
    
    #add the loadings to the grid
    loading_new_extreme = loadings.loc[w_flat,'Extreme_N'].values.reshape(w_from_latlon.shape)
    loading_new_veryhigh = loadings.loc[w_flat,'VeryHigh_N'].values.reshape(w_from_latlon.shape)
    loading_new_high = loadings.loc[w_flat,'High_N'].values.reshape(w_from_latlon.shape)
    loading_new_moderate = loadings.loc[w_flat,'Moderate_N'].values.reshape(w_from_latlon.shape)
    loading_new_low = loadings.loc[w_flat,'Low_N'].values.reshape(w_from_latlon.shape)
    
    grid['loading_new_extreme'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_extreme)
    grid['loading_new_veryhigh'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_veryhigh)
    grid['loading_new_high'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_high)
    grid['loading_new_moderate'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_moderate)
    grid['loading_new_low'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_low)
    
    #xs=np.array(xs).flatten() 
    #ys=np.array(ys).flatten()
    xs_sub = []
    ys_sub = []
    inds_row =[]
    inds_col = []

    #subset the x,y values by bounds of the multipolygon
    if poly_latlon['geometry'].values[0].geom_type=='Polygon':
        geom_bounds = poly_latlon['geometry'].values[0].bounds
        inds_row_sel,inds_col_sel = np.where((xs>geom_bounds[0])&(xs<geom_bounds[2])&
                            (ys>geom_bounds[1])&(ys<geom_bounds[3]))
        xs_sub.append(xs[inds_row_sel,inds_col_sel])
        ys_sub.append(ys[inds_row_sel,inds_col_sel])
        inds_row.append(inds_row_sel)
        inds_col.append(inds_col_sel)
            
    elif poly_latlon['geometry'].values[0].geom_type=='MultiPolygon':
        for geom in poly_latlon['geometry'].values[0].geoms:
            geom_bounds = geom.bounds
            inds_row_sel,inds_col_sel = np.where((xs>geom_bounds[0])&(xs<geom_bounds[2])&
                            (ys>geom_bounds[1])&(ys<geom_bounds[3]))
            xs_sub.append(xs[inds_row_sel,inds_col_sel])
            ys_sub.append(ys[inds_row_sel,inds_col_sel])
            inds_row.append(inds_row_sel)
            inds_col.append(inds_col_sel)
            
    xs_sub = np.concatenate(xs_sub)
    ys_sub = np.concatenate(ys_sub)
    inds_row = np.concatenate(inds_row)
    inds_col = np.concatenate(inds_col)
    
    print(len(xs.flatten()),len(xs_sub))
    tic = datetime.datetime.now()
    results = Parallel(n_jobs=14)(delayed(build_one_fccs_grid_cell)
                                 (xs_sub, ys_sub,inds_row, inds_col,kk,15) 
                                 for kk in range(len(xs_sub)))
    toc = datetime.datetime.now()
    print(toc-tic)
    #format the grid subset into a dataframs
    df_grid=gpd.GeoDataFrame(results)
    df_grid.columns = ['x_loc', 'y_loc','row','col','geometry']
    df_grid.set_geometry(col='geometry',inplace=True,crs='EPSG:5070') #need to say it's in lat/lon before transform to LCC
    df_grid=df_grid.to_crs(epsg=3347)

    
    #intersect the polygon with the grid subset
    njobs = 14
    stride = math.ceil(len(df_grid)/njobs) #round up because we drop duplicates
    tic = datetime.datetime.now()
    intersection = Parallel(n_jobs=njobs)(delayed(gpd.overlay)
                                    (df_grid.iloc[kk:kk+stride], poly, how='intersection',keep_geom_type=False)
                                     for kk in np.arange(0,len(df_grid),stride))
    toc = datetime.datetime.now()
    print(toc-tic)
    
    intersection=pd.concat(intersection).drop_duplicates()
    intersection['grid intersection area (ha)'] =intersection['geometry'].area/10000
    intersection['weights'] = intersection['grid intersection area (ha)']/intersection['fire area (ha)'] 
    intersection_xr = intersection.set_index(['row', 'col']).to_xarray()    

    grid_sub = grid.sel(rows_ctr = np.unique(intersection['row'].values),
                        cols_ctr = np.unique(intersection['col'].values))

    varis = ['day','loading_new_extreme', 'loading_new_veryhigh','loading_new_high', 'loading_new_moderate', 'loading_new_low']
    df_loading = generate_df(varis, len(poly))
    
    df_loading['day'] = poly['12Z Start Day'].values
    for var in varis[1:len(varis)]:
        df_loading[var] = np.nansum(intersection_xr['weights'].values*grid_sub[var].values)
    
    return df_loading
    

In [4]:
def build_one_fccs_grid_cell(x_location, y_location, rows, cols, index, half_cell):
    ctrx =x_location[index]
    ctry =y_location[index]
    
    sw = (ctrx-half_cell,ctry-half_cell) #SW
    se = (ctrx+half_cell,ctry-half_cell) #SE
    nw = (ctrx-half_cell,ctry+half_cell) #NW
    ne = (ctrx+half_cell,ctry+half_cell) #NE
    cell = Polygon([sw,nw,ne,se])
    return ctrx,ctry,rows[index],cols[index],cell

In [6]:
#makes and saves a geodataframe of a grid given the center and corner points for that grid as 2D matrices
def build_grid_xarray(LAT_COR, LON_COR, LAT_CTR, LON_CTR):
    #loop over the centers
    nrows_center = LAT_CTR.shape[0]
    ncols_center = LAT_CTR.shape[1]
    #print(nrows_center, ncols_center)

    nrows_corner = LAT_COR.shape[0]
    ncols_corner = LAT_COR.shape[1]
    #print(nrows_corner,ncols_corner)
    
    rows_ctr = np.arange(nrows_center)
    cols_ctr = np.arange(ncols_center)
    rows_cor = np.arange(nrows_corner)
    cols_cor = np.arange(ncols_corner)

    
    dat_grid = xr.Dataset(
        data_vars = dict(
            LAT_CTR=(['rows_ctr','cols_ctr'],LAT_CTR),
            LON_CTR=(['rows_ctr','cols_ctr'],LON_CTR),
            LAT_COR=(['rows_cor','cols_cor'],LAT_COR),
            LON_COR=(['rows_cor','cols_cor'],LON_COR),
        ),
        
        coords = dict(
            rows_ctr =(['rows_ctr'],rows_ctr),
            cols_ctr =(['cols_ctr'],cols_ctr),
            rows_cor =(['rows_cor'],rows_cor),
            cols_cor =(['cols_cor'],cols_cor),
        
        )
        
    )
    return(dat_grid)

In [7]:
#makes and saves a geodataframe of a grid given the center and corner points for that grid as 2D matrices
def build_one_gridcell(LAT_COR, LON_COR, LAT_CTR, LON_CTR, loc):
    ii=loc[0]
    jj=loc[1]

    #print(LAT_CTR[ii,jj], LON_CTR[ii,jj]) #ctr
    sw = (LON_COR[ii, jj],LAT_COR[ii, jj]) #SW
    se =(LON_COR[ii, jj+1],LAT_COR[ii, jj+1]) #SE
    nw = (LON_COR[ii+1, jj],LAT_COR[ii+1, jj]) #NW
    ne = (LON_COR[ii+1, jj+1],LAT_COR[ii+1, jj+1]) #NE
            
    poly_cell = Polygon([sw,nw,ne,se])

    return LAT_CTR[ii,jj], LON_CTR[ii,jj],ii,jj,poly_cell
    

In [8]:
#LAT and LON are 2d arrays
def calculate_grid_cell_corners(LAT, LON):
    #we will assume the very edges of the polygons don't touch the boundary of the domain
    lat_corners = (LAT[0:(LAT.shape[0]-1),  0:(LAT.shape[1])-1] + LAT[1:(LAT.shape[0]), 1:(LAT.shape[1])])/2
    lon_corners = (LON[0:(LAT.shape[0]-1),  0:(LAT.shape[1])-1] + LON[1:(LAT.shape[0]), 1:(LAT.shape[1])])/2
    return lat_corners, lon_corners


In [9]:
def generate_df(variables, length):
    df = pd.DataFrame()
    for vv in variables:
        df[vv] = np.zeros(length)
    return df

## Old code

In [ ]:
 #select the region of the fuels tiff file that's associated with the polygon
    

    print(w_flat.shape)
    #build the grid    
    rows = np.arange(win.row_off,win.row_off+win.height-1)
    cols = np.arange(win.col_off,win.col_off+win.width-1)
    COLS,ROWS=np.meshgrid(cols,rows)
    xs, ys = rasterio.transform.xy(src.transform, ROWS, COLS)
    xs=np.array(xs).flatten()
    ys=np.array(ys).flatten()
    print(xs.shape)
    

    points = [Point(xs[jj],ys[jj]) for jj in range(len(xs))]
    print(len(points))
    
    #xs_corner, ys_corner = calculate_grid_cell_corners(xs,ys)
    
    #grid = build_grid_xarray(ys_corner, xs_corner, ys, xs)
    """
    #add the loadings to the grid
    loading_new_extreme = loadings.loc[w_flat,'Extreme_N'].values.reshape(w_from_latlon.shape)
    loading_new_veryhigh = loadings.loc[w_flat,'VeryHigh_N'].values.reshape(w_from_latlon.shape)
    loading_new_high = loadings.loc[w_flat,'High_N'].values.reshape(w_from_latlon.shape)
    loading_new_moderate = loadings.loc[w_flat,'Moderate_N'].values.reshape(w_from_latlon.shape)
    loading_new_low = loadings.loc[w_flat,'Low_N'].values.reshape(w_from_latlon.shape)
    
    grid['loading_new_extreme'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_extreme)
    grid['loading_new_veryhigh'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_veryhigh)
    grid['loading_new_high'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_high)
    grid['loading_new_moderate'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_moderate)
    grid['loading_new_low'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_low)

    #print(grid)
       
    #get the locs, which are the INSIDE of the centers
    df_col = pd.DataFrame({'col_loc':grid['cols_ctr'].values[1:len(grid['cols_ctr'])-1]})
    df_row = pd.DataFrame({'row_loc':grid['rows_ctr'].values[1:len(grid['rows_ctr'])-1]})
    locs = list(product(df_row['row_loc'], df_col['col_loc']))
    print(len(locs))
    
    #make a geodataframe (in paralell of the rows and cols)
    tic = datetime.datetime.now()
    print(tic)
    results = Parallel(n_jobs=8)(delayed(build_one_gridcell)
                                 (grid['LAT_COR'].values, grid['LON_COR'].values,
                                  grid['LAT_CTR'].values, grid['LON_CTR'].values,loc) 
                                 for loc in locs)
    
    #format the grid subset into a dataframs
    df_grid=gpd.GeoDataFrame(results)
    df_grid.columns = ['lat', 'lon', 'row', 'col', 'geometry']
    df_grid.set_geometry(col='geometry',inplace=True,crs='EPSG:5070') #need to say it's in lat/lon before transform to LCC
    df_grid=df_grid.to_crs(epsg=3347)

    
    #intersect the polygon with the grid subset
    njobs = 8
    stride = math.ceil(len(df_grid)/njobs) #round up because we drop duplicates
    intersection = Parallel(n_jobs=njobs)(delayed(gpd.overlay)
                                    (df_grid.iloc[kk:kk+stride], poly, how='intersection',keep_geom_type=False)
                                     for kk in np.arange(0,len(df_grid),stride))
    
    #intersection = gpd.overlay(df_grid,poly,how='intersection',keep_geom_type=False).drop_duplicates()
    toc = datetime.datetime.now()
    print(toc)
    print(toc-tic)
    
    intersection=pd.concat(intersection).drop_duplicates()
    intersection['grid intersection area (ha)'] =intersection['geometry'].area/10000
    intersection['weights'] = intersection['grid intersection area (ha)']/intersection['fire area (ha)'] 
    intersection_xr = intersection.set_index(['row', 'col']).to_xarray()
    
    grid_sub = grid.sel(rows_ctr = np.unique(intersection['row'].values),
                        cols_ctr = np.unique(intersection['col'].values))

    varis = ['day','loading_new_extreme', 'loading_new_veryhigh','loading_new_high', 'loading_new_moderate', 'loading_new_low']
    df_loading = generate_df(varis, len(poly))
    
    df_loading['day'] = poly['12Z Start Day'].values
    for var in varis[1:len(varis)]:
        df_loading[var] = np.nansum(intersection_xr['weights'].values*grid_sub[var].values)
    
    return df_loading"""

In [ ]:
from shapely.geometry import Point
import geopandas as gpd

d = {'geometry': [Point(1, 2), Point(2, 1)]}

gdf = gpd.GeoDataFrame(d)
print(gdf.has_sindex)

index = gdf.sindex
print(gdf)

In [ ]:
#load in the merra grid
grid = xr.open_dataset('HRRR_GRID.nc')

#get the bounds of the buffered polygons
poly_latlon =fire_daily.iloc[0:1].to_crs(epsg=4326)
print(poly_latlon)
bounds = poly_latlon.buffer(1).bounds

#first check for rows and cols
[rows,cols] = np.where((grid.LAT_CTR.values>bounds['miny'].values)&
                (grid.LAT_CTR.values<bounds['maxy'].values)&
                (grid.LON_CTR.values>bounds['minx'].values)&
                (grid.LON_CTR.values<bounds['maxx'].values))
locs = zip(rows,cols)
     
#make a geodataframe (in paralell of the rows and cols)
results = Parallel(n_jobs=8)(delayed(build_one_gridcell)
                            (grid['LAT_COR'].values, grid['LON_COR'].values,
                            grid['LAT_CTR'].values, grid['LON_CTR'].values,loc) 
                            for loc in locs)
    
#format the grid subset into a dataframs
df_grid=gpd.GeoDataFrame(results)
df_grid.columns = ['lat', 'lon', 'row', 'col', 'geometry']
df_grid.set_geometry(col='geometry',inplace=True,crs='EPSG:4326') #need to say it's in lat/lon before transform to LCC
#df_grid=df_grid.to_crs(epsg=3347)

tic = datetime.datetime.now()
index_grid=df_grid.sindex
index_poly = poly_latlon.sindex
toc=datetime.datetime.now()
print(toc-tic)

intersection = gpd.overlay(df_grid,poly_latlon,how='intersection',keep_geom_type=False).drop_duplicates()
print(intersection)    

#make it into boxes
#do a contains?"""